In [11]:
import os
import warnings
import copy

import uuid
import random
import itertools

import networkx as nx

import math
import pandas as pd
import numpy as np
from scipy.stats import skewnorm

import sys
sys.path.insert(1, '../../scripts/')
from simulation.graphs import graph_generator as gg_

In [12]:
class LR():
    '''object to store metadata and relevant information for the ligan-receptor dimension of tensor
    for internal use
    
    '''
    def __init__(self, B, ligands, receptors, edge_list, network_type = None, alpha = None, 
                 fit = None, comp = None, p = None):
        '''Initialize
        
        Parameters
        ----------

        B: nx.Graph
            a undirected bipartite network representing PPI between ligands and receptors (direction would always be L-->R)
        ligands: list 
            ligand IDs for each protein
        receptors: list
            receptor IDs for each protein
        self.edge_list: list
            each entry is a tuple representing a potential interaction between a ligand-receptor pair, ligands on 0 index of each tuple
        network_type: str
            "scale-free" indicates scale-free network, "normal" indicates a normal degree distribution
        alpha: float
            scale-free exponent for network degree distribution (recommended 2<alpha<3)
        p: float
            probability of adding an edge when using network_type option = 'normal'
        fit: igraph.FittedPowerLaw
            scale-free network parameters for B_ig (p-value from Kolmogrov-Smirnov test)
        comp: pd.DataFrame or None
            summary of differences in network properties between  bipartite network and similar Barabasi network
        '''
        if network_type is None:
            self.network_type = 'user-speficied'
        else:
            self.network_type = network_type
        self.B = B
        self.ligands = ligands
        self.receptors = receptors
        self.edge_list = edge_list
        self.alpha = alpha
        self.fit = fit
        self.comp = comp
        self.p = p

In [80]:
def weight_bias(n, skew):
    '''
    n: int
        Number of entries to bin
    skew: extent to which skew binning
    '''
    unbiased = [1/n]*n
    if skew == 0:
        biased = np.array(unbiased)
    elif skew == 1:
        biased = [0]*len(unbiased)
        biased[-1] = 1
        biased = np.array(biased)
    else:
        X = np.linspace(0, len(unbiased), len(unbiased))
        biased = skewnorm.pdf(X, a = 1, loc = len(unbiased), scale = (1-skew/1)*len(unbiased))
        biased = biased/sum(biased)
    return biased*100

class CCI_MD():
    '''Generate the CCI network for the tensor slice at time point 0'''
    
    def cci_network(self, n_cells, directional = True):
            '''Initialize the cell-cell interaction network.

            n_cells: int
                the total number of cells to simulate; all cell-cell pairs will have a potential interaction, but only
                those that actually interact will have a score > 0 in the tensor slice
            directional: bool
                whether cell-cell interactions are directional (tuple of cell (A,B) indicates interaction from A-->B) or 
                not

            '''
            # generate random cell ids
            self.cell_ids = [str(uuid.uuid4()).split('-')[-1] for i in range(n_cells)]
            if directional:
                self.cell_interactions = list(itertools.permutations(self.cell_ids, 2))
            else:
                self.cell_interactions = list(itertools.combinations(self.cell_ids, 2))
    def generate_metadata(self, n_cell_cats = {2: 0}, cat_skew = 0, 
                         remove_homotypic = None):
        '''Generate metadata groupings for the cells (individual) and proteins (as ligand-receptor pairs). Categories are defined as distinct types of 
        metadata associated with the cell or protein, e.g. "cell type" and "cell cycle phase" for cells. Subcategories are
        the associated labels within a category, e.g. "T-cell" and "dendritic cell" within the "cell type" category.
        
        Note: For skew, 0 means evenly distributed, 1 means all cells fall into the first category/subcategory. 
        
        n_cell_cats: dict
            The length of the dictionary represents the total number of categories associated with the cell
            Each key is an integer representing the number of subcategories for the particular category. 
            Each value is a float [0,1] indicating the skew of distribution of cells across 
            subcategories within each category. 
        cat_skew: float [0,1]
            Skew of distribution of cells across categories
        remove_homotypic: int
            whether to remove homotypic ineractions between cells by cell category; how many categories to consider; 
            must be <= the number of categories present in the metadata

        '''
        if remove_homotypic > len(n_cell_cats):
            raise ValueError('The value for "remove_homotypic" cannot be larger than the total number of categories associated with the cells')
        
        # generate cell categories
        cell_categories = [str(uuid.uuid4()).split('-')[-1] for i in range(len(n_cell_cats))]

        # group each cell into the categories above
        # generate categories
        cell_categories = [str(uuid.uuid4()).split('-')[-1] for i in range(len(n_cell_cats))]
        cat_bias = weight_bias(n = len(cell_categories), skew = 0)
        self.cell_ids = pd.DataFrame(data = {'cell_id': self.cell_ids, 
                        'category': random.choices(population = cell_categories, weights=cat_bias, 
                                                  k=len(self.cell_ids))})
        
        # generate subcategories
        self.cell_ids['subcategory'] = float('nan')
        i = 0
        for n_subcat, subcat_skew in n_cell_cats.items():
            sub = self.cell_ids[self.cell_ids.category == cell_categories[i]]
            subcat_bias = weight_bias(n = n_subcat, skew = subcat_skew)
            self.cell_ids.loc[sub.index, 'subcategory'] = random.choices(population = [str(uuid.uuid4()).split('-')[-1] for i in range(n_subcat)], 
                           weights=subcat_bias, k=sub.shape[0])
            i += 1
        self.cell_metadata = self.cell_ids
        del self.cell_ids
        

        if remove_homotypic is not None and remove_homotypic > 0: # remove homotypic interactions of a given category
            print('Remove homotypic cell interactions for {} categories'.format(remove_homotypic))
            i = 0
            to_remove = list()
            while i < remove_homotypic:
                cat = cell_categories[i]
                sub = self.cell_metadata[self.cell_metadata.category == cell_categories[i]]
                cell_ids = sub.cell_id.tolist()

                for ccp in self.cell_interactions:
                    sub_ = sub[(sub.cell_id == ccp[0]) | (sub.cell_id == ccp[1])]
                    if sub_.shape[0] == 2 and sub_.subcategory.unique().shape[0] == 1:
                        to_remove.append(ccp)

                i += 1

            # remove homotypic interactions as identified above for categories 1-i
            self.cell_interactions = list(set(self.cell_interactions).difference(to_remove))

            # filter out any cells that no longer are present 
            cell_ids = list(set(sum(list(zip(*self.cell_interactions)), ())))
            self.cell_metadata = self.cell_metadata[self.cell_metadata.cell_id.isin(cell_ids)]
            self.cell_metadata.reset_index(inplace = True, drop = True)



In [3]:
class Simulate():
    def __init__(self):
        '''Initialize self

        '''
    
    def LR_network(self, network_type = None, B = None, subset = False, **params):
        '''
        Simulates a PPI network of *potential* ligand-receptor interactions, or extracts information. \
        from a use provided network.Defines one tensor dimension
        Caveats: for a scale-free network, the number of ligands = the number of receptors \
                 for either network, there may be disconnected edges depending on "p"
        
        Parameters
        ----------
        network_type: str
             "scale-free" to generate a scale-free network or "normal" to generate a network with a normal degree distribution
        B: nx.Graph
            a user provided undirected, unweighted bipartite network. Assumes in B.nodes, ligands are listed \
            before receptors. Takes precedence over network_type.
        subset: bool
            if B is provided and subset is true, this will take a random subset of the network, dropping disconnected nodes \
            (of a specified size, specfied in params)
        **params: dict (keys for each option specified below)
            the required parameters for generating a bipartite, undirected random network either scale-free or not. \
            
            network_type = scale-free: keys - nodes, degrees, alpha, edges (see graphs.graph_generator.bipartite_sf for description) 
            network_type = normal: keys - n_ligands, n_receptors, p analogous to n,m,p in nx.bipartite.gnmk_random_graph
            B != None: keys - n_ligands as described above
            subset = True: keys - 
                n_ligands as described above
                'subset_size' a value between (0,1) indicating the proportional \
                size of the subset (by nodes) compared to the network
                'subset_type' either 'edges' or 'nodes' indicating whether to subset by removing nodes or edges \
                (edges recommended because they maintain the scale-free property)
        
        Returns
        ----------
        self.LR: 
            populates LR object, key outputs outlined here
        self.LR.B: nx.Graph
            undirected bipartite graph with specified degree distribution (power or normal), or user specified B \
            disconnected nodes are removed
        self.LR.edge_list: list
            each entry is a tuple representing a potential interaction between a ligand-receptor pair, ligands on 0 index of each tuple


        '''
        gg = gg_() # return networkx object for graphs
        user = False
        if B is not None: #untested
            user = True
            # properties checked when calling gg.nx_to_edgelist
            if network_type is not None:
                warnings.warn('You have specified a network type and provided a network, B will take priority over network type')
            if 'n_ligands' not in params:
                raise ValueError('For a provided B, you must specify n_ligands in params')
            
            if subset:
                if 'subset_size' not in params or 'subset_type' not in params:
                    raise ValueError('To subset B, you must provide a desired subset_size and subset_type')
                if params['subset_type'] == 'edges':
                    B = gg.subset_edges(B, subset_size = params['subset_size'], drop = True)
                elif params['subset_type'] == 'nodes': 
                    B = gg.subset_nodes(B, subset_size = params['subset_size'], drop = True)
                else:
                    raise ValueError("The subset_type param must be either 'edges' or 'nodes'")
            
            
        elif network_type == 'scale-free': 
            if 'degrees' not in params or 'nodes' not in params:
                raise ValueError('Must specify degrees and nodes in **params')
            if 'alpha' not in params: 
                params['alpha'] = 2 # also default in gg obj, didn't make it a **kwrag
            if 'edges' not in params:
                B, node_groups, fit, comp = gg.bipartite_sf(nodes = params['nodes'], degrees = params['degrees'], 
                                                            alpha = params['alpha'])
            else:
                B, node_groups, fit, comp = gg.bipartite_sf(nodes = params['nodes'], degrees = params['degrees'], 
                                                        alpha = params['alpha'], edges = ['edges'])  
            B = B['nx']
            params['n_ligands'] = params['nodes'] # same no. of ligands and receptors
        elif network_type == 'normal':
            if sorted(params) != ['n_ligands', 'n_receptors', 'p']:
                raise ValueError('Must specify n_ligands, n_receptors in **params')
            else:
                B = nx.bipartite.random_graph(params['n_ligands'],params['n_receptors'], params['p'])
        else:
            raise ValueError('Must specify an appropriate network_type or provide a network B')
        
        B, edge_list, ng = gg.nx_to_edgelist(B, params['n_ligands']) # formatting/extract info
        
        # store PPI information in LR()
        if user:
            self.LR = LR(B, ng['1'], ng['2'], edge_list)
        elif network_type == 'scale-free':
            self.LR = LR(B, ng['1'], ng['2'], edge_list, network_type = network_type, 
                         alpha = params['alpha'], fit = fit, comp = comp)
        elif network_type == 'normal':
            self.LR = LR(B, ng['1'], ng['2'], edge_list, network_type = network_type, p = params['p'])

    def emulate_sf_network(self, G):
        '''Emulate a user-provided (recommended scale-free) network for L-R pair tensors dimension
        
        Parameters
        ----------
        G: nx.Graph
            user-provided network (recommended scale-free)
        
        Returns
        ---------
        G2: nx.Graph
            random bipartite scale-free network built using G's properties
        
        '''
        gg = gg_()
        fit = gg.power_fit(G)
        if fit.p < 0.05:
            warnings.warn('Input network is not scale-free')
        print('----Simulated network------')
        
        G2, node_groups, fit2, comp = gg.bipartite_sf(nodes = round(len(G.nodes)), # should be 1/2 the nodes, but many are disconnected 
                                 degrees = np.median([i[1] for i in G.degree]),
                                 alpha = fit.alpha, edges = len(G.edges),
                                 check_properties = True, compare_barabasi = False)
        G2 = G2['nx']
        gg.drop_disconnected_nodes(G2)
        return G2
    
    
    
    def tensor_slice_t0(self, binary = True):

        '''Simulates a static time point tensor slice
        
        Parameters
        ----------
        binary: bool
            whether L-R scores are binary or continuous b/w [0,1]
        
        Returns
        -------
        self.ts: pd.DataFrame
            matrix with cell network_type pairs as columns, ligand-receptor pairs as rows, scores as entries
        '''
        
        # TO DO: 
        # first separate into groupings/metadata covariates?
        # generate biased distributions of LR scores
        self.ts = pd.DataFrame(columns = list(itertools.combinations(range(n_cells), 2)), index = self.LR.edge_list)
        
    def copy(self):
        return copy.deepcopy(self)

In [42]:
# output = tempfile.mkstemp(suffix = '_bipartite_sf.csv', dir = './')[1]
# __location__ = os.path.realpath(
#     os.path.join(os.getcwd(), os.path.dirname(os.path.basename(output))))

In [5]:
# init
sim_sf = Simulate() 
sim_norm = Simulate()

# simulate a randomly connected ligand-receptor network (potential interactions)
sim_sf.LR_network(network_type = 'scale-free', **{'nodes': 10, 'degrees': 1, 'alpha': 2}) #scale-free
sim_norm.LR_network(network_type = 'normal', **{'n_ligands': 10, 'n_receptors': 5, 'p': 0.5}) # normally distributed




/home/hratch/Projects/cci_dt/notebooks/simulation/tmpqrip2ql3_bipartite_sf.csv
Generate undirected, bipartite, scale-free graph
Check network properties
All properties are as expected


../../scripts/simulation/graphs.py:164: UserWarning: 9 nodes are disconnected, removing from network
  warnings.warn(mssg)


In [ ]:
# cell metadata
cci = CCI_MD()
cci.cci_network(n_cells = 100, directional = False)
cci.generate_metadata(n_cell_cats = {2: 0, 3: 0.3}, cat_skew = 0, remove_homotypic = 1)

Remove homotypic cell interactions for 1 categories
